In [31]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.optimize import minimize
from statsmodels.gam.api import GLMGam, BSplines
from statsmodels.genmod.families import Gamma as GammaFamily
from statsmodels.genmod.families import Binomial
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [15]:
data = pd.read_csv("./weather_data_filled_1950_2023.csv.gz", 
                compression='gzip', 
                sep=',',  
                encoding='utf-8', 
                low_memory=False)

In [16]:
data.describe()

,NUM_POSTE,LAT,LON,ALTI,RR,TNTXM,day_of_year,year
count,5.650137e+06,5.650137e+06,5.650137e+06,5.650137e+06,5.641423e+06,5.650137e+06,5.650137e+06,5.650137e+06
mean,3.309204e+07,4.742934e+01,1.699505e+00,1.450522e+02,1.956111e+00,1.139960e+01,1.831341e+02,1.989083e+03
std,9.268499e+06,5.888690e-01,7.028758e-01,6.095975e+01,4.374759e+00,6.568808e+00,1.054492e+02,2.024369e+01
min,1.800300e+07,4.642533e+01,1.078330e-01,3.100000e+01,0.000000e+00,-2.030000e+01,1.000000e+00,1.950000e+03
25%,2.807000e+07,4.697183e+01,1.178333e+00,1.070000e+02,0.000000e+00,6.600000e+00,9.200000e+01,1.973000e+03
50%,3.613900e+07,4.733617e+01,1.692333e+00,1.320000e+02,0.000000e+00,1.146893e+01,1.830000e+02,1.992000e+03
75%,4.105000e+07,4.788667e+01,2.243833e+00,1.690000e+02,1.900000e+00,1.650000e+01,2.740000e+02,2.006000e+03
max,4.534000e+07,4.880150e+01,3.053333e+00,4.620000e+02,1.808000e+02,3.280000e+01,3.660000e+02,2.023000e+03


In [17]:
data.head()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,RR,TNTXM,day_of_year,year,is_imputed
0,18003002,LES AIX,47.216667,2.55,182,0.0,-1.106477,1,1950,True
1,18003002,LES AIX,47.216667,2.55,182,1.6,-3.019302,2,1950,True
2,18003002,LES AIX,47.216667,2.55,182,5.4,3.601141,3,1950,True
3,18003002,LES AIX,47.216667,2.55,182,2.0,7.346058,4,1950,True
4,18003002,LES AIX,47.216667,2.55,182,1.9,7.248367,5,1950,True


#### N'utilisons qu'une portion des stations :

In [14]:
data.NOM_USUEL.value_counts().sort_index()

NOM_USUEL
AIGURANDE                26663
ALLAINES                  2525
AMBOISE                  21610
AMBOISE - LYCEE          10852
AMILLY                   21367
                         ...  
VILLEMURLIN-PEUPLIERS    21488
VILLENY                  26936
VOUVRAY                   6661
VOVES                    16313
YZEURES/CREUSE            2306
Name: count, Length: 462, dtype: int64

In [18]:
df = data[data['NOM_USUEL'].str.startswith(('A', 'B', 'C'))]

In [24]:
df.count()

NUM_POSTE      1619497
NOM_USUEL      1619497
LAT            1619497
LON            1619497
ALTI           1619497
RR             1619133
TNTXM          1619497
day_of_year    1619497
year           1619497
is_imputed     1619497
dtype: int64

### 3.2. Marginal Modeling :

In [45]:
class MarginalPrecipitationModel:
    """
    Modèle marginal en trois étapes pour les précipitations extrêmes
    basé sur Zhong et al. (2025)
    """
    
    def __init__(self, df, threshold_prob=0.9):
        """
        Parameters:
        -----------
        df : DataFrame
            Données avec colonnes: RR, TNTXM, day_of_year, year, LAT, LON, ALTI, NUM_POSTE
        threshold_prob : float
            Probabilité de seuil pour la distribution Gamma (défaut: 0.9)
        """
        self.df = df.copy()
        self.threshold_prob = threshold_prob
        
        # Préparer les données (enlever les valeurs manquantes et extrêmes)
        self.df_valid = self.df[
            (self.df['RR'].notna()) & 
            (self.df['RR'] > 0) & 
            (self.df['RR'] < 1000)
        ].copy()
        
        # Normaliser l'altitude en km
        self.df_valid['ALTI_km'] = self.df_valid['ALTI'] / 1000
        
        # Résultats des modèles
        self.gamma_model = None
        self.binomial_model = None
        self.gpd_params = None
        
    def fit_gamma_bulk(self, k_day=10, k_spatial=10, k_alti=10):
        """
        Étape 1: Ajuster une distribution Gamma pour la masse de la distribution
        
        Modèle: RR ~ TNTXM + s(day_of_year) + te(LON, LAT) + s(ALTI)
        """
        print("Étape 1: Ajustement du modèle Gamma...")
        
        # Préparer les splines
        x_spline_day = BSplines(
            self.df_valid['day_of_year'], 
            df=[k_day], 
            degree=[3]
        )
        
        x_spline_alti = BSplines(
            self.df_valid['ALTI_km'], 
            df=[k_alti], 
            degree=[3]
        )
        
        # Pour le terme spatial te(LON, LAT), on utilise des splines tensorielle
        # Approximation avec produit de splines univariées
        x_spline_lon = BSplines(
            self.df_valid['LON'], 
            df=[k_spatial], 
            degree=[3]
        )
        
        x_spline_lat = BSplines(
            self.df_valid['LAT'], 
            df=[k_spatial], 
            degree=[3]
        )
        
        # Créer la matrice de design
        import statsmodels.api as sm
        
        # Modèle linéaire pour la température
        X_temp = sm.add_constant(self.df_valid['TNTXM'])
        
        # Combiner avec les splines
        X_full = np.column_stack([
            X_temp,
            x_spline_day.basis,
            x_spline_lon.basis,
            x_spline_lat.basis,
            x_spline_alti.basis
        ])
        
        # Ajuster le modèle Gamma avec lien log
        self.gamma_model = sm.GLM(
            self.df_valid['RR'], 
            X_full,
            family=sm.families.Gamma(link=sm.families.links.log())
        ).fit()
        
        # Prédictions
        fitted_mean = self.gamma_model.fittedvalues
        
        # Estimer les paramètres de la Gamma
        # Pour Gamma: E[Y] = shape * scale, Var[Y] = shape * scale^2
        # Donc: shape = E[Y]^2 / Var[Y], scale = Var[Y] / E[Y]
        residuals = self.df_valid['RR'] - fitted_mean
        
        # Utiliser la déviance pour estimer le paramètre de dispersion
        scale_est = self.gamma_model.scale
        shape_est = 1 / scale_est
        scale_param = fitted_mean / shape_est
        
        # Calculer les quantiles à 90%
        self.df_valid['gamma_shape'] = shape_est
        self.df_valid['gamma_scale'] = scale_param
        self.df_valid['threshold'] = stats.gamma.ppf(
            self.threshold_prob, 
            a=shape_est, 
            scale=scale_param
        )
        
        print(f"  Paramètre de forme moyen: {shape_est:.4f}")
        print(f"  Seuil moyen (90%): {self.df_valid['threshold'].mean():.2f} mm")
        
        return self.gamma_model
    
    def fit_binomial_exceedance(self, k_day=10, k_spatial=10, k_alti=10):
        """
        Étape 2: Ajuster un modèle binomial pour les dépassements de seuil
        
        Modèle: I(RR > threshold) ~ TNTXM + s(day_of_year) + te(LON, LAT) + s(ALTI)
        """
        print("\nÉtape 2: Ajustement du modèle binomial...")
        
        # Créer la variable binaire
        self.df_valid['exceeds_threshold'] = (
            self.df_valid['RR'] > self.df_valid['threshold']
        ).astype(int)
        
        # Préparer les splines (même structure que Gamma)
        x_spline_day = BSplines(
            self.df_valid['day_of_year'], 
            df=[k_day], 
            degree=[3]
        )
        
        x_spline_alti = BSplines(
            self.df_valid['ALTI_km'], 
            df=[k_alti], 
            degree=[3]
        )
        
        x_spline_lon = BSplines(
            self.df_valid['LON'], 
            df=[k_spatial], 
            degree=[3]
        )
        
        x_spline_lat = BSplines(
            self.df_valid['LAT'], 
            df=[k_spatial], 
            degree=[3]
        )
        
        import statsmodels.api as sm
        X_temp = sm.add_constant(self.df_valid['TNTXM'])
        
        X_full = np.column_stack([
            X_temp,
            x_spline_day.basis,
            x_spline_lon.basis,
            x_spline_lat.basis,
            x_spline_alti.basis
        ])
        
        # Ajuster le modèle binomial avec lien logit
        self.binomial_model = sm.GLM(
            self.df_valid['exceeds_threshold'],
            X_full,
            family=sm.families.Binomial()
        ).fit()
        
        self.df_valid['exceedance_prob'] = self.binomial_model.fittedvalues
        
        n_exceedances = self.df_valid['exceeds_threshold'].sum()
        print(f"  Nombre de dépassements: {n_exceedances}")
        print(f"  Probabilité moyenne de dépassement: {self.df_valid['exceedance_prob'].mean():.4f}")
        
        return self.binomial_model
    
    def fit_gpd_tail(self, k_day=10, k_spatial=10, k_alti=10):
        """
        Étape 3: Ajuster une distribution de Pareto généralisée (GPD) 
        pour les excès au-dessus du seuil
        
        Modèle: 
        - log(scale) ~ log(threshold) + TNTXM + s(day_of_year) + te(LON, LAT) + s(ALTI)
        - shape: constant
        """
        print("\nÉtape 3: Ajustement du modèle GPD...")
        
        # Sélectionner uniquement les excès
        df_exceedances = self.df_valid[
            self.df_valid['exceeds_threshold'] == 1
        ].copy()
        
        # Calculer les excès
        df_exceedances['excess'] = (
            df_exceedances['RR'] - df_exceedances['threshold']
        )
        
        # Vérifier qu'on a des excès positifs
        df_exceedances = df_exceedances[df_exceedances['excess'] > 1e-5].copy()
        
        print(f"  Nombre d'excès: {len(df_exceedances)}")
        
        # Préparer les covariables
        x_spline_day = BSplines(
            df_exceedances['day_of_year'], 
            df=[k_day], 
            degree=[3]
        )
        
        x_spline_alti = BSplines(
            df_exceedances['ALTI_km'], 
            df=[k_alti], 
            degree=[3]
        )
        
        x_spline_lon = BSplines(
            df_exceedances['LON'], 
            df=[k_spatial], 
            degree=[3]
        )
        
        x_spline_lat = BSplines(
            df_exceedances['LAT'], 
            df=[k_spatial], 
            degree=[3]
        )
        
        import statsmodels.api as sm
        
        # Inclure log(threshold) comme offset/variable
        X_predictors = np.column_stack([
            np.ones(len(df_exceedances)),  # Intercept
            np.log(df_exceedances['threshold']),
            df_exceedances['TNTXM'],
            x_spline_day.basis[:, 1:],
            x_spline_lon.basis[:, 1:],
            x_spline_lat.basis[:, 1:],
            x_spline_alti.basis[:, 1:]
        ])
        
        scaler = StandardScaler()
        X_predictors[:, 1:] = scaler.fit_transform(X_predictors[:, 1:])

        cond_number = np.linalg.cond(X_predictors)
        print(f"Condition number de la matrice X: {cond_number:.2e}")
        if cond_number > 1e10:
            print("ATTENTION: Matrice mal conditionnée (colinéarité). L'optimisation va échouer.")

        # Pour la GPD, on utilise une approche de maximum de vraisemblance
        # On modélise log(scale) avec GLM Gamma
        
        def gpd_neg_loglik(params, y, X):
            """Negative log-likelihood for GPD"""
            n_beta = X.shape[1]
            beta = params[:n_beta]
            xi = params[n_beta]  # shape parameter
            
            log_scale = X @ beta
            scale = np.exp(log_scale)
            
            # GPD log-likelihood
            if abs(xi) < 1e-5:
            # Cas exponentiel (limite xi -> 0)
                ll = -np.sum(np.log(scale)) - np.sum(y / scale)
            else:
                z = 1 + xi * y / scale
                if np.any(z <= 0):
                    return 1e12
            
            # Log-vraisemblance standard
            ll = -np.sum(np.log(scale)) - (1/xi + 1) * np.sum(np.log(z))

            return -ll
        
        # Initialisation
        initial_beta = np.zeros(X_predictors.shape[1])
        initial_beta[0] = np.log(df_exceedances['excess'].mean())
        initial_xi = 0.1
        initial_params = np.concatenate([initial_beta, [initial_xi]])
        
        bounds = [(None, None)] * X_predictors.shape[1] + [(-0.5, 0.8)]

        # Optimisation
        result = minimize(
            gpd_neg_loglik,
            initial_params,
            args=(df_exceedances['excess'].values, X_predictors),
            method='L-BFGS-B',
            bounds=bounds,
            options={'maxiter': 5000, 'xatol': 1e-4, 'fatol': 1e-4}
        )
        
        if result.success:
            beta_hat = result.x[:-1]
            xi_hat = result.x[-1]
            
            self.gpd_params = {
                'beta': beta_hat,
                'xi': xi_hat,
                'X_predictors': X_predictors
            }
            
            # Calculer les échelles ajustées
            log_scale_fitted = X_predictors @ beta_hat
            df_exceedances['gpd_scale'] = np.exp(log_scale_fitted)
            df_exceedances['gpd_shape'] = xi_hat
            
            print(f"  Paramètre de forme ξ: {xi_hat:.4f}")
            print(f"  Échelle moyenne: {df_exceedances['gpd_scale'].mean():.2f}")
            
            # Stocker les résultats
            self.df_exceedances = df_exceedances
        else:
            print("  ATTENTION: L'optimisation GPD n'a pas convergé")
            self.gpd_params = None
        
        return self.gpd_params
    
    def transform_to_uniform(self):
        """
        Transformer les données vers une échelle uniforme [0,1]
        en utilisant la transformation intégrale de probabilité.
        Version corrigée : utilise pd.Series indexée pour éviter IndexError.
        """
        print("\nTransformation vers échelle uniforme...")

        # Créer une Series indexée comme self.df_valid
        uniform_scores = pd.Series(index=self.df_valid.index, dtype=float)

        # Indices non-excédants
        non_exceed_idx = self.df_valid['exceeds_threshold'] == 0
        if non_exceed_idx.any():
            # stats.gamma.cdf peut accepter tableaux alignés
            uniform_scores.loc[non_exceed_idx] = stats.gamma.cdf(
                self.df_valid.loc[non_exceed_idx, 'RR'].values,
                a=self.df_valid.loc[non_exceed_idx, 'gamma_shape'].values,
                scale=self.df_valid.loc[non_exceed_idx, 'gamma_scale'].values
            )

        # Pour les excédants (si GPD ajusté)
        if self.gpd_params is not None and hasattr(self, 'df_exceedances'):
            exceed_idx = self.df_valid['exceeds_threshold'] == 1
            if exceed_idx.any():
                # Probabilité d'excéder le seuil pour ces observations
                exceed_prob = self.df_valid.loc[exceed_idx, 'exceedance_prob']

                # On utilise df_exceedances (index commun) pour récupérer scale et xi
                # itération sur df_exceedances est sûre car ses index appartiennent à self.df_valid.index
                for orig_idx, row_exc in self.df_exceedances.iterrows():
                    # excess = observed - threshold (déjà calculé dans df_exceedances)
                    excess = row_exc['excess']
                    scale = row_exc['gpd_scale']
                    xi = row_exc['gpd_shape']

                    # CDF de la GPD conditionnelle P(Y <= y | Y > u)
                    if abs(xi) < 1e-6:
                        p_excess = 1 - np.exp(-excess / scale)
                    else:
                        z = 1 + xi * excess / scale
                        if z > 0:
                            p_excess = 1 - z ** (-1.0 / xi)
                        else:
                            p_excess = 1.0

                    # Probabilité totale: P(Y <= y) = (1 - p_u) + p_u * P_cond
                    pu = exceed_prob.loc[orig_idx]
                    uniform_scores.loc[orig_idx] = (1.0 - pu) + pu * p_excess

        # Remplir les éventuels NaN restants à sécurité (par ex. si aucune catégorie)
        uniform_scores = uniform_scores.fillna(0.0)

        # Empêcher la valeur exactement égale à 1.0
        self.df_valid['uniform_score'] = (uniform_scores / (1.0 + 1e-10)).astype(float)

        print(f"  Scores uniformes calculés pour {self.df_valid['uniform_score'].notna().sum()} observations")

        return self.df_valid['uniform_score']

    
    def transform_to_unit_pareto(self):
        """
        Transformer vers échelle Pareto unitaire pour l'analyse de dépendance
        """
        if 'uniform_score' not in self.df_valid.columns:
            self.transform_to_uniform()
        
        # Transformation: Y_pareto = 1 / (1 - U)
        # où U ~ Uniform(0,1)
        unit_pareto = 1 / (1 - self.df_valid['uniform_score'])
        self.df_valid['unit_pareto'] = unit_pareto
        
        print(f"\nTransformation Pareto unitaire:")
        print(f"  Min: {unit_pareto.min():.2f}")
        print(f"  Max: {unit_pareto.max():.2f}")
        print(f"  Médiane: {unit_pareto.median():.2f}")
        
        return unit_pareto
    
    def fit_complete_model(self):
        """
        Ajuster le modèle complet en trois étapes
        """
        self.fit_gamma_bulk()
        self.fit_binomial_exceedance()
        self.fit_gpd_tail()
        self.transform_to_uniform()
        self.transform_to_unit_pareto()
        
        return self
    
    def get_results_summary(self):
        """
        Obtenir un résumé des résultats
        """
        summary = {
            'n_observations': len(self.df_valid),
            'n_exceedances': self.df_valid['exceeds_threshold'].sum(),
            'threshold_90pct': self.df_valid['threshold'].mean(),
            'gamma_shape': self.df_valid['gamma_shape'].mean(),
            'exceedance_prob': self.df_valid['exceedance_prob'].mean(),
        }
        
        if self.gpd_params is not None:
            summary['gpd_shape'] = self.gpd_params['xi']
            summary['gpd_scale_mean'] = self.df_exceedances['gpd_scale'].mean()
        
        return summary

In [46]:
# Exemple d'utilisation
if __name__ == "__main__":
    model = MarginalPrecipitationModel(df, threshold_prob=0.9)
    model.fit_complete_model()
    
    # Obtenir les résultats
    summary = model.get_results_summary()
    print("\nRésumé des résultats:")
    for key, value in summary.items():
         print(f"  {key}: {value}")
    
    # Les données transformées sont dans model.df_valid
    # avec les colonnes 'uniform_score' et 'unit_pareto'
    
    pass

Étape 1: Ajustement du modèle Gamma...
  Paramètre de forme moyen: 0.6685
  Seuil moyen (90%): 11.46 mm

Étape 2: Ajustement du modèle binomial...
  Nombre de dépassements: 68891
  Probabilité moyenne de dépassement: 0.0975

Étape 3: Ajustement du modèle GPD...
  Nombre d'excès: 68891
Condition number de la matrice X: 2.27e+01
  Paramètre de forme ξ: 0.0370
  Échelle moyenne: 6.02

Transformation vers échelle uniforme...
  Scores uniformes calculés pour 706454 observations

Transformation Pareto unitaire:
  Min: 1.04
  Max: 4796810.74
  Médiane: 2.01

Résumé des résultats:
  n_observations: 706454
  n_exceedances: 68891
  threshold_90pct: 11.460000930116331
  gamma_shape: 0.6685482901542256
  exceedance_prob: 0.0975166111310887
  gpd_shape: 0.03695875743213462
  gpd_scale_mean: 6.023142051712279


### 3.3 Dependance Modeling